In [88]:
# Se importan las librerías

import cv2
import math
import pathlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10,6]

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

import warnings 
warnings.filterwarnings('ignore')

**U.A**: Inteligencia artificial y Redes Neuronales
**Proyecto**: Reconocimiento Facial


In [72]:
print("El equipo esta conformador por:")
equipo_IA=("René Emiliano Cerda Medellín", "Lerin Alejandro Hernández","Oscar Marcelo Fragoso","Milton Leonardo Rodríguez","Yair Obed Morales Ortíz")
integrantes_map={}
for i in range(len(equipo_IA)):
    integrantes_map[i]=equipo_IA[i]
integrantes_map

In [73]:
# Se definene las carpetas con cada dataset

Train_dir = pathlib.Path('../input/avengers-faces-dataset/images/train')
Test_dir = pathlib.Path('../input/avengers-faces-dataset/images/test')
Val_dir = pathlib.Path('../input/avengers-faces-dataset/images/val')

image_count_train = len(list(Train_dir.glob('*/*.png')))
print("Training Samples ---> ",image_count_train)

image_count_test = len(list(Test_dir.glob('*/*.png')))
print("Testing Samples ---> ",image_count_test)

image_count_val = len(list(Val_dir.glob('*/*.png')))
print("Testing Samples ---> ",image_count_val)

In [75]:
#Variables a utilizar
random_seed=123
batch_size = 32
img_height = 180
img_width = 180

In [76]:
# Set de entrenamiento
train_ds = tf.keras.preprocessing.image_dataset_from_directory(Train_dir,
                                                               seed=random_seed,
                                                               image_size=(img_height, img_width),
                                                               batch_size=batch_size)

# Set de prueba
test_ds = tf.keras.preprocessing.image_dataset_from_directory(Test_dir,
                                                               seed=random_seed,
                                                               image_size=(img_height, img_width),
                                                               batch_size=batch_size)

# Set de validación
val_ds = tf.keras.preprocessing.image_dataset_from_directory(Val_dir,
                                                               seed=random_seed,
                                                               image_size=(img_height, img_width),
                                                               batch_size=batch_size)

In [77]:
# Se enlistan las etiquetas de las imagenes, en este caso tenemos 5

class_names = train_ds.class_names
class_map={}
name_class_map={"chris_evans":"Chris_Evans", "chris_hemsworth":"Chris_Hemsworth", "mark_ruffalo":"Mark_Ruffalo", 
"robert_downey_jr":"Robert_Downey_Jr", "scarlett_johansson":"Scarlett_Johansson"}

for i in range(len(class_names)):
    class_map[i]=class_names[i]
class_map

In [78]:
#Gráfica con la distribución de las etiquetas de las imagenes

class_count = []
for i in range(len(class_names)):
    class_count.append(len(list(Train_dir.glob('{}/*.png'.format(class_names[i])))))

    
plt.pie(class_count, shadow=True, startangle=0, explode=[0,0,0,0,0.1], autopct='%1.1f%%', counterclock=False, labels=train_ds.class_names)
#, labels=class_map.values())
plt.show()

In [79]:
#Se observa un arreglo de 25 imagenes del dataset
plt.figure(figsize=[14,14])
for image, label in train_ds.take(1):
    for e,i in enumerate(image):
        if e<25:
            plt.subplot(5,5,e+1)
            plt.imshow(i.numpy().astype('uint8'))
            plt.title(name_class_map[class_map[label[e].numpy()]])
            plt.xticks([])
            plt.yticks([])
plt.show()

In [80]:
#Se observa un arreglo de 2 imagenes del dataset aleatorio
plt.figure(figsize=[14,14])
for image, label in train_ds.take(1):
    for e,i in enumerate(image):
        if e<2:
            plt.subplot(2,2,e+1)
            plt.imshow(i.numpy().astype('uint8'))
            plt.title(name_class_map[class_map[label[e].numpy()]])
            plt.xticks([])
            plt.yticks([])
plt.show()

In [81]:
# Se mejora el rendimiento del entrenamiento y además se mezcla las imagenes de entrenamiento

AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [82]:
#Se desarrolla el modelo Sequential de tensorflow keras agregando varias capas

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(img_height, img_width,3)),
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(16, (3, 3), padding="same", activation="relu" ),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    # tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32, (3, 3), padding="same", activation="relu" ),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    # tf.keras.layers.BatchNormalization(),
    # tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv2D(64, (3, 3), padding="same", activation="relu" ),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    # tf.keras.layers.BatchNormalization(),
    # tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    # tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    # tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(5, activation='softmax')
])

model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
             optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
             metrics=['accuracy'])

model.summary()

In [83]:
# Se entrena el modelo con 25 épocas

chkpt = ModelCheckpoint('checkpoints/best_model.h5', monitor='val_loss', mode='min', save_best_only=True, verbose=1)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5, min_delta=0.01)

epochs=25
history_1 = model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=[es, chkpt])

In [84]:
#Porcentaje de Precisión
results = model.evaluate(train_ds, verbose=0)

print("    Loss: {:.4f}".format(results[0]))
print("Accuracy: {:.2f}%".format(results[1] * 100))

In [85]:
#Se visualiza el desarrollo del entrenamiento graficando las epocas vs las pérdidas y la exactitud

acc_train = history_1.history['accuracy']
acc_val = history_1.history['val_accuracy']

loss_train = history_1.history['loss']
loss_val = history_1.history['val_loss']

epochs = range(1,len(loss_train)+1)
plt.figure(figsize=[15,5])

plt.subplot(1,2,1)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1,2,2)
plt.plot(epochs, acc_train, 'g', label='Training accuracy')
plt.plot(epochs, acc_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [86]:
# Se guarda el modelo para poderse cargar directamente a partir de un "checkpoint"

saved_model = tf.keras.models.load_model('checkpoints/best_model.h5')
saved_model

In [87]:
#Se realizan predicciones a cierto número de imagenes y se compara cada su etiqueta actual
#con la predecida

test_batch = [(x, y) for (x,y) in val_ds.take(1)] # loading 1 batch of data
image, label = test_batch[0][0], test_batch[0][1]

prediction = saved_model.predict(image)
prediction = np.argmax(prediction, axis=1)


plt.figure(figsize=[14,14])
for e in range(20):
    plt.subplot(5,5,e+1)
    plt.imshow(image[e].numpy().astype('uint8'))
    actual_label = name_class_map[class_map[label[e].numpy()]]
    prediction = saved_model.predict(np.expand_dims(image[e], axis=0))
    predicted_label = name_class_map[class_map[np.argmax(prediction)]]
    #predicted_label = np.argmax(saved_model.predict(image))
    plt.title('Actual:  {}\nPredicted: {}'.format(actual_label, predicted_label))
    plt.xticks([])
    plt.yticks([])
plt.tight_layout()
plt.show()